In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import util
from sort.sort import *
# from util import get_car, read_license_plate,
from utils.utils import get_car,read_license_plate

Using CPU. Note: This module is much faster with a GPU.


In [2]:
# Yolo model to detect cars
coco_model = YOLO('yolov8n.pt')
# Load model for license plate detector
license_plate_detector = YOLO('./models/license_plate_detector.pt')

# Object Tracker:
mot_tracker = Sort()

In [ ]:
# Read the Video
cap = cv2.VideoCapture('./data/sample.mp4')

#Class ids for vehicles:
vehicle_classes = [2,3,5,7]
# Car,bike,truck

In [ ]:
# Read frames from the video:
results = {}
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        # Detect Vehicles
        detections = coco_model(frame)[0]
        vehicle_detections = list()
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,conf_score,class_id = detection
            if(class_id in vehicle_classes): 
                vehicle_detections.append(detection) # Bounding boxes of the car detections
    # Track the vehicles using track ids
    track_ids = mot_tracker.update(np.asarray(vehicle_detections))
    # Detect License Plates
    license_plates = license_plate_detector(frame)[0]
    for license_plate in license_plates.boxes.data.tolist():
        x1,y1,x2,y2,conf_score,class_id = license_plate
        # Assign license plate to the car
        xcar1,ycar1,xcar2,ycar2,car_id = get_car(license_plate,track_ids) # Which license plate belongs to which car
        # Crop the License Plate
        license_plate_cropped = frame[int(y1):int(y2),int(x1):int(x2),:]
        # Process this License Plate (BGR)
        license_plate_grey = cv2.cvtColor(license_plate_cropped,cv2.COLOR_BGR2GRAY)
        # Thresholding
        _, license_plate_grey_thres = cv2.threshold(license_plate_grey, 64, 255, cv2.THRESH_BINARY_INV)
        #cv2.imshow('grey_threshold',license_plate_grey_thres)

        license_plate_text, license_plate_text_score = read_license_plate(license_plate_grey_thres)
        #print(license_plate_text)
        if(license_plate_text is not None):
            key = f'Frame:{frame_nmr}; CarID:{car_id}'

            results[key] = {

                'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                'license_plate': {'bbox': [x1, y1, x2, y2],
                'text': license_plate_text,
                'bbox_score': conf_score,
                'text_score': license_plate_text_score}

                }

In [ ]:
# import json
# with open('results.json','w') as f:
#     json.dump(results,f)

import json
with open('results.json','r') as f:
    results = json.load(f)

In [ ]:
def prepare_output_df(results):
    df = pd.DataFrame(results).T
    df['index'] = df.index
    df = df.reset_index(drop=True)
    df = df.rename(columns = {0:'main_dict'})
    df['car'] = df['main_dict'].apply(lambda x:x['car'])
    df['license_plate'] = df['main_dict'].apply(lambda x:x['license_plate'])

    df['index'] = df['index'].apply(lambda x: x.split(';'))
    df['frame_nmr'] = df['index'].apply(lambda x:x[0].split(':')[1]).apply(lambda x:int(x))
    df['car_id'] = df['index'].apply(lambda x:x[1].split(':')[1]).apply(lambda x: int(float(x)))

    df['car_bbox'] = df['car'].apply(lambda x: x['bbox'])
    df['License_plate_bbox'] = df['license_plate'].apply(lambda x: x['bbox'])
    df['license_number'] = df['license_plate'].apply(lambda x: x['text'])
    df['license_number_score'] = df['license_plate'].apply(lambda x: x['text_score'])
    
    df = df[['frame_nmr','car_id','car_bbox','License_plate_bbox','license_number','license_number_score']].reset_index(drop=True)
    return df

In [ ]:
output_df = prepare_output_df(results)

In [ ]:
# output_df.to_csv('output.csv')

In [3]:
output_df = pd.read_csv('output.csv')

In [4]:
from post_processing.interpolate_missing_data import interpolate_bounding_boxes

In [5]:
interpolated_data = interpolate_bounding_boxes(output_df)

TypeError: string indices must be integers

In [6]:
data = output_df.copy(deep=True)

In [26]:
frame_numbers = np.array([row for row in data['Frame_number']])
car_ids = np.array([row for row in data['Car_ID']])
car_bboxes = [np.array(row) for row in data['Car_BBOX']]

In [27]:
car_bboxes

[array('[752.8723754882812, 1369.210693359375, 1430.7803955078125, 1982.91064453125]', dtype='<U76'),
 array('[2197.398982088146, 1154.2834652275594, 2785.5585232511503, 1736.5472034973143]', dtype='<U79'),
 array('[752.7638437200508, 1369.2583936107194, 1430.4360306214135, 1982.8932146936904]', dtype='<U79'),
 array('[2201.178783562989, 1163.453317068365, 2783.907437479453, 1740.8521179919255]', dtype='<U77'),
 array('[749.0850529066378, 1380.2561231106586, 1430.1694234887354, 2000.7944594150042]', dtype='<U79'),
 array('[2196.6997749906327, 1171.529181511491, 2784.9978363487658, 1751.7260273511752]', dtype='<U79'),
 array('[2194.0966464246176, 1174.1965844449346, 2789.7639641510996, 1755.6260221622583]', dtype='<U80'),
 array('[2189.132944580224, 1176.7711775547352, 2789.446821516278, 1762.0336419513103]', dtype='<U78'),
 array('[2190.6484422351814, 1175.744386353321, 2792.2373173040232, 1764.4548165910885]', dtype='<U79'),
 array('[737.1985902865017, 1396.050406032306, 1425.60826919

In [23]:
car_bboxes

['Unnamed: 0', 'Frame_number', 'Car_ID', 'Car_BBOX', 'License_Plate_BBOX']